The Apache dataset contains the files [access.log](https://github.com/gdv/foundationsCS/raw/main/students/ex-data/apache/access.log) and [error.log](https://github.com/gdv/foundationsCS/raw/main/students/ex-data/apache/error.log) that contains the logfile of the accesses to a web server and the errors.
The *access.log* is in [Common Log Format](https://en.wikipedia.org/wiki/Common_Log_Format).
The entries in *error.log* usually have a corresponding entry in *access.log*

## Read the file *access.log*. This file is not trivial to read correctly.

Hint: The first row of the file *access.log* does not contain the names of the columns: we can use the `names` option.

Hint: We use a custom separator, otherwise the fields `type`, `url`, and `prot` would be combined together.

## Count the number of accesses (number of lines) made by an IP number

## Count the number of successful accesses (status 200) made by an IP number

## Count the number of accesses for each directory served

## For each origin, count the number of successful accesses

## For each origin, count the number of unsuccessful accesses, split according to the status code

## From the results of the previous point, add a column with the error class (the first digit of the status code)

## Cluster the accesses in 5-minutes time slices (e.g. from 14:00 to 14:05, from 14:05 to 14:10, etc). Count the number of accesses for each time slice